# CareerCon 2019
Okay so I am entering this competitition so I will be making a detailed documentation as well. Let's first see what our data looks like and do some simple data visualization.

In [13]:
#Imports
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from tensorflow.python.keras.layers import Dense, BatchNormalization
from tensorflow.python.keras.models import Sequential

In [14]:
X = pd.read_csv("./Data/X_train.csv")
y = pd.read_csv("./Data/y_train.csv")
X_test = pd.read_csv("./Data/X_test.csv")

In [15]:
display(X.head())
print(X.shape)

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


(487680, 13)


In [16]:
display(y.head())
print(y.shape)

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


(3810, 3)


Lets combine X_train and y_train dataframes together.

In [17]:
data = X.copy()

In [18]:
for i in range(3810):
    if i % 1000 == 0 and i != 0:
        print("Finished {}/{} series".format(i,3810))
    data.loc[data['series_id'] == i, 'surface'] = y.loc[y['series_id'] == i, 'surface'].values[0]
print("Complete")

Finished 1000/3810 series
Finished 2000/3810 series
Finished 3000/3810 series
Complete


In [19]:
#A little feature engineering, include the magnitude for orientation, velocity and acceleration
data['orientation_mag'] = np.sqrt(np.square(data['orientation_X']) + np.square(data['orientation_Y']) + np.square(data['orientation_Z']))
data['angular_velocity_mag'] = np.sqrt(np.square(data['angular_velocity_X']) + np.square(data['angular_velocity_Y']) + np.square(data['angular_velocity_Z']))
data['linear_acceleration_mag'] = np.sqrt(np.square(data['linear_acceleration_X']) + np.square(data['linear_acceleration_Y']) + np.square(data['linear_acceleration_Z']))

#Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,surface,orientation_mag,angular_velocity_mag,linear_acceleration_mag
0,2821_53,2821,53,-0.26148,0.95358,0.142700,-0.044207,0.051148,-0.035061,0.023948,-2.70410,2.5519,-8.9533,fine_concrete,0.999024,0.066475,9.694634
1,532_27,532,27,-0.95456,0.25756,0.030872,-0.146700,0.099754,0.118090,-0.150450,-0.28532,7.8315,-6.2200,wood,0.989179,0.215711,10.005109
2,2325_121,2325,121,-0.76254,0.62947,0.090446,-0.118870,0.072278,-0.027139,-0.025409,-0.83923,4.3535,-9.3531,soft_pvc,0.992915,0.081279,10.350737
3,1491_37,1491,37,0.91779,0.36814,0.059764,0.136220,0.058134,-0.030917,0.049745,0.19611,3.3452,-9.1789,soft_tiles,0.990675,0.082523,9.771439
4,2709_111,2709,111,-0.55733,0.81622,0.123480,-0.089009,0.026779,0.136970,-0.522140,-2.07620,5.8291,-8.9658,fine_concrete,0.996032,0.540470,10.893786


In [42]:
#Is the data skewed? Lets see
data['surface'].value_counts()

concrete                  99712
soft_pvc                  93696
wood                      77696
tiled                     65792
fine_concrete             46464
hard_tiles_large_space    39424
soft_tiles                38016
carpet                    24192
hard_tiles                 2688
Name: surface, dtype: int64

In [20]:
#One hot encode the labels and store as numpy array
y = []
for surface in data['surface'].values:
    if surface == 'carpet':
        y.append([1,0,0,0,0,0,0,0,0])
    elif surface == 'concrete':
        y.append([0,1,0,0,0,0,0,0,0])
    elif surface == 'fine_concrete':
        y.append([0,0,1,0,0,0,0,0,0])
    elif surface == 'hard_tiles':
        y.append([0,0,0,1,0,0,0,0,0])
    elif surface == 'hard_tiles_large_space':
        y.append([0,0,0,0,1,0,0,0,0])
    elif surface == 'soft_pvc':
        y.append([0,0,0,0,0,1,0,0,0])
    elif surface == 'soft_tiles':
        y.append([0,0,0,0,0,0,1,0,0])
    elif surface == 'tiled':
        y.append([0,0,0,0,0,0,0,1,0])
    elif surface == 'wood':
        y.append([0,0,0,0,0,0,0,0,1])
y = np.array(y)
y.shape

(487680, 9)

In [21]:
X = data.drop(['surface','row_id','series_id', 'measurement_number'], axis=1).values
X.shape

(487680, 13)

In [22]:
#Standardize the training data before using model
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [23]:
model = Sequential()
model.add(Dense(512, input_dim=13, activation='relu'))
model.add(Dense(512,  activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(9, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
model.fit(X,y, epochs=10, batch_size=256, validation_split=0.25)

Train on 365760 samples, validate on 121920 samples
Epoch 1/10
365760/365760 [==============================] - 28s 78us/sample - loss: 0.6284 - acc: 0.7559 - val_loss: 0.5181 - val_acc: 0.7874
Epoch 2/10
365760/365760 [==============================] - 28s 78us/sample - loss: 0.4257 - acc: 0.8308 - val_loss: 0.4127 - val_acc: 0.8373
Epoch 3/10
365760/365760 [==============================] - 29s 78us/sample - loss: 0.3461 - acc: 0.8626 - val_loss: 0.3043 - val_acc: 0.8776
Epoch 4/10
365760/365760 [==============================] - 28s 77us/sample - loss: 0.2967 - acc: 0.8822 - val_loss: 0.2984 - val_acc: 0.8809
Epoch 5/10
365760/365760 [==============================] - 28s 77us/sample - loss: 0.2641 - acc: 0.8965 - val_loss: 0.2843 - val_acc: 0.8878
Epoch 6/10
365760/365760 [==============================] - 28s 78us/sample - loss: 0.2347 - acc: 0.9077 - val_loss: 0.2827 - val_acc: 0.8913
Epoch 7/10
365760/365760 [==============================] - 28s 78us/sample - loss: 0.2122 - acc

Nice! So we achieved an accuracy of 93% on the validation set. Lets just double check our model is actually performing this well with some cross-validation.

# Model Evaluation

In [41]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4)
kf.get_n_splits(X, y)
i = 0
print("Performing Cross Validation...")
for train_index, test_index in kf.split(X, y):
    i+=1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train, epochs=10, batch_size=256, verbose=0)
    evaluation = model.evaluate(X_test, y_test, verbose=0)
    print("Cross-val score #{}: accuracy = {}".format(i, evaluation[1]))
print("Done Cross Validation!")

Performing Cross Validation...
Cross-val score #1: accuracy = 0.945275604724884
Cross-val score #2: accuracy = 0.9669537544250488
Cross-val score #3: accuracy = 0.9750245809555054
Cross-val score #4: accuracy = 0.9546424150466919
Done Cross Validation!


In [58]:
#Precision and Recall scores
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

target_names = ['carpet','concrete','fine_concrete',
                'hard_tiles','hard_tiles_large_space','soft_pvc',
                'soft_tiles','tiled','wood']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model.fit(X_train,y_train, epochs=10, batch_size=256, verbose=0)
predictions = model.predict_classes(X_test)
labels = np.argmax(y_test,axis=1)
print(classification_report(labels, predictions, target_names = target_names))

                        precision    recall  f1-score   support

                carpet       0.99      0.96      0.97      5953
              concrete       0.97      0.98      0.97     25029
         fine_concrete       0.98      0.99      0.98     11623
            hard_tiles       0.98      0.99      0.99       672
hard_tiles_large_space       0.98      0.98      0.98      9847
              soft_pvc       0.99      0.98      0.99     23300
            soft_tiles       0.99      1.00      0.99      9397
                 tiled       0.97      0.99      0.98     16547
                  wood       0.98      0.96      0.97     19552

             micro avg       0.98      0.98      0.98    121920
             macro avg       0.98      0.98      0.98    121920
          weighted avg       0.98      0.98      0.98    121920



Okay, so our model seems to be pretty good! Even though the dataset is uneven. All thats left is to use the model and submit our predictions!